In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from scipy.stats import *

from ipywidgets import interact, interactive, fixed, widgets

from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
output_notebook()

Loading BokehJS ...

In [2]:
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv("regLinPoli.csv")

In [4]:
def agregar_unos(m):
    normalizado = StandardScaler().fit_transform(m)
    unos = np.array([np.ones(normalizado.shape[0])])
    return np.concatenate((unos.T,normalizado),axis=1)

def sets(m):
    datos = agregar_unos(m)
    entrenamiento, prueba = train_test_split(datos, test_size=0.2)#, random_state=18)
    return entrenamiento, prueba

def separados(m):
    entrenamiento, prueba = sets(m)
    
    entrenamiento_x = entrenamiento[:,range(0,entrenamiento.shape[1]-1)]
    entrenamiento_y = entrenamiento[:,entrenamiento.shape[1]-1]

    prueba_x = prueba[:,range(0,prueba.shape[1]-1)]
    prueba_y = prueba[:,prueba.shape[1]-1]
    return entrenamiento_x, entrenamiento_y, prueba_x, prueba_y

def sep_lambda(m):
    entrenamientos_x, entrenamientos_y, prueba_x, prueba_y = separados(m)
    corte = int(round(0.8*len(entrenamientos_x)))
    entrenamiento_x = entrenamientos_x[0:corte,:]
    entrenamiento_y = entrenamientos_y[0:corte]
    lamb_x = entrenamientos_x[corte:len(entrenamientos_x),:]
    lamb_y = entrenamientos_y[corte:len(entrenamientos_y)]
    return entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, lamb_x, lamb_y

In [5]:
def LMS(df,eta):
    """
    Función de eta
    """
    
    lamb = 0
    
    entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, lamb_x, lamb_y = sep_lambda(df)
        
    w = np.ones(df.shape[1])

    error = []    

    contador = 0

    while (sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2 + lamb*np.dot(w.T,w)> 0.01:
        error.append((sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2 + lamb*np.dot(w.T,w))
        w[0] = w[0] + (eta * sum((entrenamiento_y - np.dot(w,entrenamiento_x.T)))/len(entrenamiento_y))
        for i in range(1,len(w)):
            w[i] = w[i] + (eta * sum((entrenamiento_y - np.dot(w,entrenamiento_x.T)) #
                                     * entrenamiento_x[:,i])/len(entrenamiento_y)) - (lamb * np.dot(w.T,w))
        if (sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2 + lamb*np.dot(w.T,w) > error[len(error)-1]:
            break
        contador += 1
    
    error_prueba = (sum(np.dot(w,prueba_x.T) - prueba_y))**2 + lamb*np.dot(w.T,w)
    
    return error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba, lamb_x, lamb_y

In [6]:
def regularizacion(df,eta=0.05,paso=0.0001220703125):
    error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba, lamb_x, lamb_y = LMS(df,eta)
    W = np.copy(w)
    lamb = 0
    errorl = []
    primero = (sum(np.dot(w,lamb_x.T) - lamb_y))**2 + lamb*np.dot(w.T,w)
    for j in np.arange(0,1,paso):
        eta = 0.05
        lamb = j
        w[0] = w[0] + (eta * sum((lamb_y - np.dot(w,lamb_x.T)))/len(lamb_y))
        for i in range(1,len(w)):
            w[i] = w[i] + (eta * sum((lamb_y - np.dot(w,lamb_x.T)) #
                                     * lamb_x[:,i])/len(lamb_y)) - (lamb * np.dot(w.T,w))
        errorl.append((sum(np.dot(w,lamb_x.T) - lamb_y))**2 + lamb*np.dot(w.T,w))
        if (sum(np.dot(w,lamb_x.T) - lamb_y))**2 + lamb*np.dot(w.T,w) > primero:
            break
    return prueba_x, prueba_y, w, errorl, W, lamb_x, lamb_y

def minimo(df,eta=0.05,paso=0.0001220703125):
    prueba_x, prueba_y, w, errorl, W, lamb_x, lamb_y = regularizacion(df,eta,paso)
    a = 0
    lambdas = np.arange(0,1,paso)[0:len(errorl)]
    for i in range(0,len(errorl)):
        if errorl[i] == min(errorl):
            a = i
    error_minimo = errorl[a]
    lambdita_minimo = lambdas[a]
    return error_minimo, lambdita_minimo, prueba_x, prueba_y, w, errorl, lambdas, W, lamb_x, lamb_y

In [7]:
def graficar(df, eta=0.05,paso=0.0001220703125):
    
    error_minimo, lambdita_minimo, prueba_x, prueba_y, w, errorl, lambdas, W, lamb_x, lamb_y = minimo(df,eta,paso)
    
    s1 = figure(width=750, plot_height=500, title=None)

    s1.circle(lambdas,errorl[0:len(errorl)], size=5, color="red", alpha=0.5)
    s1.circle(lambdita_minimo,error_minimo,size=6, color="blue",alpha=0.9)
    s1.title.text = "Errores respecto al parámetro λ"
    s1.title.text_font_size = "25px"
    s1.title.align = "center"
    s1.xaxis.axis_label = "λ"
    s1.yaxis.axis_label = "Errores"
    
    show(s1)
    
    print('w: {}\n').format(w)
    print('lambda óptimo: {}\n').format(lambdita_minimo)
    print('error mínimo con el set de lambdas: {}\n').format(error_minimo)
    print('error con set de prueba: {}\n').format((sum(np.dot(w,prueba_x.T) - prueba_y))**2)
    print('error con set de lambdas antes\n de la regularización: {}\n').format((sum(np.dot(W,lamb_x.T) - lamb_y))**2)

In [8]:
graficar(df, eta=0.01)

w: [ 0.03677605  0.22142715  0.58327919 -0.53855512 -0.16923688]

lambda óptimo: 0.0045166015625

error mínimo con el set de lambdas: 0.560822725488

error con set de prueba: 5.50648540247

error con set de lambdas antes
 de la regularización: 4.51078652262

